In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf

spark=SparkSession.builder.appName("aggregate").getOrCreate()

data=[[2,"Alice"],[5,"Bob"]]
schema=("age INT, Name STRING")
df=spark.createDataFrame(data,schema=schema)
df.show()
df_1=df.agg(sf.min(df.age))
df_2=df.agg({"age":"max"})
df_1.show()
df_2.show()



+---+-----+
|age| Name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+

+--------+
|min(age)|
+--------+
|       2|
+--------+

+--------+
|max(age)|
+--------+
|       5|
+--------+




Running Sum in window function 

In [0]:
import pyspark
from pyspark.sql import *
#from pyspark.sql.functions import col, sum
spark=SparkSession.builder.appName("running_total").getOrCreate()


data=[["1","Alice","80"],
      ["2","Bob","90"],
      ["3","Alice","85"],
      ["4","Bob","70"],
      ["5","Carol","75"]]
schema=("id string, name String, score string")

df= spark.createDataFrame(data,schema=schema)
df.show()
windowSpec=Window.orderBy("id")
df.select("id","score",sum(col("score")).over(windowSpec).alias("running_total")).show()

+---+-----+-----+
| id| name|score|
+---+-----+-----+
|  1|Alice|   80|
|  2|  Bob|   90|
|  3|Alice|   85|
|  4|  Bob|   70|
|  5|Carol|   75|
+---+-----+-----+

+---+-----+-------------+
| id|score|running_total|
+---+-----+-------------+
|  1|   80|         80.0|
|  2|   90|        170.0|
|  3|   85|        255.0|
|  4|   70|        325.0|
|  5|   75|        400.0|
+---+-----+-------------+



In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

spark=SparkSession.builder.appName("running_total").getOrCreate()


data=[[1,"Alice",80],
      [2,"Bob",90],
      [3,"Alice",85],
      [4,"Bob",70],
      [5,"Carol",75]]
schema=("id int, name String, score int")

df= spark.createDataFrame(data,schema=schema)
df.show()
windowSpec=Window.orderBy("id")
df.select("id","score",sum("score").over(windowSpec).alias("running_total")).show()

+---+-----+-----+
| id| name|score|
+---+-----+-----+
|  1|Alice|   80|
|  2|  Bob|   90|
|  3|Alice|   85|
|  4|  Bob|   70|
|  5|Carol|   75|
+---+-----+-----+

+---+-----+-------------+
| id|score|running_total|
+---+-----+-------------+
|  1|   80|           80|
|  2|   90|          170|
|  3|   85|          255|
|  4|   70|          325|
|  5|   75|          400|
+---+-----+-------------+



Distinct and Count 

In [0]:
import pyspark 
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Distinct").getOrCreate()

data=[[14,"Tom"],
      [23,"Alice"],
      [23,"Alice"]]
schema="age INT, name STRING"
df=spark.createDataFrame(data,schema=schema).distinct()
df.count() #better to count in next line#


2

In [0]:
df.show()

+---+-----+
|age| name|
+---+-----+
| 14|  Tom|
| 23|Alice|
+---+-----+



                                                           DROP 

In [0]:

import pyspark 
from pyspark.sql import *
from pyspark.sql.functions import *
spark=SparkSession.builder.appName("Drop").getOrCreate()

data=[[14,"Tom"],
[23,"Alice"],
[16,"Bob"]]

schema="age INT, name STRING"

df=spark.createDataFrame(data,schema)
df2=spark.createDataFrame([Row(height=80,name="Tom"),Row(height=85,name="Bob")])

df.drop('name').show()

df.drop(df.age).show()



+---+
|age|
+---+
| 14|
| 23|
| 16|
+---+

+-----+
| name|
+-----+
|  Tom|
|Alice|
|  Bob|
+-----+



In [0]:
#Cannot drop col(‘name’) due to ambiguous reference.

df.join(df2,df.name==df2.name,'inner').drop(col(df.name)).sort('age').show()
# Error AnalysisException: [AMBIGUOUS_REFERENCE] Reference `name` is ambiguous, could be: [`name`, `name`].

---------------------------------------------------------------------------
PySparkTypeError                          Traceback (most recent call last)
File <command-1195613495446551>, line 3
      1 #drop the colun that joined both dataframe 
----> 3 df.join(df2,df.name==df2.name,'inner').drop(col(df.name)).sort('age').show()
      4 # Error AnalysisException: [AMBIGUOUS_REFERENCE] Reference `name` is ambiguous, could be: [`name`, `name`].

File /databricks/spark/python/pyspark/sql/utils.py:162, in try_remote_functions.<locals>.wrapped(*args, **kwargs)
    160     return getattr(functions, f.__name__)(*args, **kwargs)
    161 else:
--> 162     return f(*args, **kwargs)

File /databricks/spark/python/pyspark/sql/functions.py:224, in col(col)
    197 @try_remote_functions
    198 def col(col: str) -> Column:
    199     """
    200     Returns a :class:`~pyspark.sql.Column` based on the given column name.
    201 
   (...)
    222     Column<'x'>
    223     """
--> 224     return _invo

In [0]:
#drop the colun that joined both dataframe 

df.join(df2,df.name==df2.name,'inner').drop('name').sort('age').show()

+---+------+
|age|height|
+---+------+
| 14|    80|
| 16|    85|
+---+------+



In [0]:
df4=df.withColumn('a.b.c',lit(1))
df4.show()

+---+-----+-----+
|age| name|a.b.c|
+---+-----+-----+
| 14|  Tom|    1|
| 23|Alice|    1|
| 16|  Bob|    1|
+---+-----+-----+



In [0]:
df4.drop('a.b.c').show()

+---+-----+
|age| name|
+---+-----+
| 14|  Tom|
| 23|Alice|
| 16|  Bob|
+---+-----+



In [0]:
#Cannot find a column matching the expression “a.b.c”.
df4.drop(col("a.b.c")).show()

+---+-----+-----+
|age| name|a.b.c|
+---+-----+-----+
| 14|  Tom|    1|
| 23|Alice|    1|
| 16|  Bob|    1|
+---+-----+-----+



                                          DROP DUPLICATES

In [0]:
import pyspark
from  pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *

df=spark.createDataFrame([
Row(name='Alice',age=5,height=80),
Row(name='Alice',age=5,height=80),
Row(name='Alice',age=10,height=80)
])
df.show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice|  5|    80|
|Alice| 10|    80|
+-----+---+------+



In [0]:
df.dropDuplicates(['age','height']).show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice|  5|    80|
|Alice| 10|    80|
+-----+---+------+

